# Import necessary libraries

## Pip install

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!pip install datasets evaluate sacrebleu transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 21.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB

## Others

In [3]:
import json
import os
import pandas as pd
import numpy as np
import torch

import torch.nn as nn
import torch.optim as optim
from torch.utils import data

import math

In [4]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import Dataset

In [5]:
%cd "/content/drive/MyDrive/ML/MachineTranslation"

/content/drive/MyDrive/ML/MachineTranslation


In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Config

In [47]:
max_token_length = 128
sample_used = 10000
epochs = 5

batch_size = 64
learning_rate = 2e-5
weight_decay = 1e-3

embeddings_size = 300
hidden_size = 256
num_layers = 2
dropout_rate = .5

d_model = 512
n_head = 8
factor = 4

# Data preprocessing

## Preprocessing

In [8]:
path_train_data_en = os.path.join(os.getcwd(), "train.en")
path_train_data_vi = os.path.join(os.getcwd(), "train.vi")

with open(path_train_data_en, "r") as file:
    text = file.readlines()
    text = [line.strip() for line in text]
    df_train_en = pd.DataFrame(text, columns=["english"])

with open(path_train_data_vi, "r") as file:
    text = file.readlines()
    text = [line.strip() for line in text]
    df_train_vn = pd.DataFrame(text, columns=["vietnamese"])

df_train = pd.concat([df_train_vn, df_train_en], axis=1)
df_train

,vietnamese,english
0,Khoa học đằng sau một tiêu đề về khí hậu,Rachel Pike : The science behind a climate hea...
1,"Trong 4 phút , chuyên gia hoá học khí quyển Ra...","In 4 minutes , atmospheric chemist Rachel Pike..."
2,Tôi muốn cho các bạn biết về sự to lớn của nhữ...,I &apos;d like to talk to you today about the ...
3,Có những dòng trông như thế này khi bàn về biế...,Headlines that look like this when they have t...
4,Cả hai đều là một nhánh của cùng một lĩnh vực ...,They are both two branches of the same field o...
...,...,...
133312,Tôi muốn kết luận rằng hành động của hàng ngàn...,I want to end by saying it &apos;s been the ac...
133313,Rất cảm ơn đã lắng nghe .,Thank you very much for your time .
133314,,
133315,Paul Pholeros : Làm sao để bớt nghèo khổ ? Hãy...,Didier Sornette : How we can predict the next ...


In [9]:
path_val_data_en = os.path.join(os.getcwd(), "tst2012.en")
path_val_data_vi = os.path.join(os.getcwd(), "tst2012.vi")

with open(path_val_data_en, "r") as file:
    text = file.readlines()
    text = [line.strip() for line in text]
    df_val_en = pd.DataFrame(text, columns=["english"])

with open(path_val_data_vi, "r") as file:
    text = file.readlines()
    text = [line.strip() for line in text]
    df_val_vn = pd.DataFrame(text, columns=["vietnamese"])

df_val = pd.concat([df_val_vn, df_val_en], axis=1)
df_val

,vietnamese,english
0,Làm sao tôi có thể trình bày trong 10 phút về ...,How can I speak in 10 minutes about the bonds ...
1,Câu chuyện này chưa kết thúc .,This is not a finished story .
2,Nó là một trò chơi ghép hình vẫn đang được xếp .,It is a jigsaw puzzle still being put together .
3,Hãy để tôi kể cho các bạn về vài mảnh ghép nhé .,Let me tell you about some of the pieces .
4,Hãy tưởng tượng mảnh đầu tiên : một người đàn ...,Imagine the first piece : a man burning his li...
...,...,...
1548,Đây không phải là vấn đề giữa quyền riêng tư v...,This is not a question between privacy against...
1549,Đây là một vấn đề giữa tự do và sự kiểm soát .,It &apos;s a question of freedom against contr...
1550,Và trong khi chúng ta vẫn tin tưởng vào chính ...,And while we might trust our governments right...
1551,"Và liệu chúng ta có còn tin tưởng , một cách m...","And do we trust , do we blindly trust , any fu..."


In [10]:
path_test_data_en = os.path.join(os.getcwd(), "tst2013.en")
path_test_data_vi = os.path.join(os.getcwd(), "tst2013.vi")

with open(path_test_data_en, "r") as file:
    text = file.readlines()
    text = [line.strip() for line in text]
    df_test_en = pd.DataFrame(text, columns=["english"])

with open(path_test_data_vi, "r") as file:
    text = file.readlines()
    text = [line.strip() for line in text]
    df_test_vn = pd.DataFrame(text, columns=["vietnamese"])

df_test = pd.concat([df_test_vn, df_test_en], axis=1)
df_test

,vietnamese,english
0,"Khi tôi còn nhỏ , Tôi nghĩ rằng BắcTriều Tiên ...","When I was little , I thought my country was t..."
1,Tôi đã rất tự hào về đất nước tôi .,And I was very proud .
2,"Ở trường , chúng tôi dành rất nhiều thời gian ...","In school , we spent a lot of time studying th..."
3,Mặc dù tôi đã từng tự hỏi không biết thế giới ...,Although I often wondered about the outside wo...
4,"Khi tôi lên 7 , tôi chứng kiến cảnh người ta x...","When I was seven years old , I saw my first pu..."
...,...,...
1263,"Tôi thực sự tin , nếu ta coi người khác như nh...","I truly believe , if we can see one another as..."
1264,Những tấm hình không phải là về bản thân vấnđề...,These images are not of issues . They are of p...
1265,Không có ngày nào mà tôi không nghĩ về những n...,There is not a day that goes by that I don &ap...
1266,Tôi hi vọng những tấm hình sẽ đánh thức một ng...,I hope that these images awaken a force in tho...


In [11]:
hf_train = Dataset.from_dict({
    "english": df_train["english"],
    "vietnamese": df_train["vietnamese"]
})

hf_val = Dataset.from_dict({
    "english": df_val["english"],
    "vietnamese": df_val["vietnamese"]
})

hf_test = Dataset.from_dict({
    "english": df_test["english"],
    "vietnamese": df_test["vietnamese"]
})

In [12]:
import random

random.seed(42)

# Sample 500 random indices
random_indices = random.sample(range(len(hf_train)), sample_used)

# Select the samples using the indices
hf_train = hf_train.select(random_indices)

In [13]:
from transformers import AutoTokenizer

checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [14]:
vocab_dict = tokenizer.get_vocab()

In [15]:
source_lang = "english"
target_lang = "vietnamese"
prefix = "translate English to Vietnamese: "


def preprocess_function(examples):
    print(examples)
    inputs = [example for example in examples[source_lang]]
    targets = [example for example in examples[target_lang]]
    model_inputs = tokenizer(inputs, text_target=targets, padding="max_length", max_length=max_token_length, truncation=True)
    return model_inputs

In [16]:
hf_val_tokenized = hf_val.map(preprocess_function, batched=True)
hf_train_tokenized = hf_train.map(preprocess_function, batched=True)
hf_test_tokenized = hf_test.map(preprocess_function, batched=True)

Output hidden; open in https://colab.research.google.com to view.

In [17]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint, padding="max_length", max_length=max_token_length)

## Data Loader

In [18]:
class CustomDataset(data.Dataset):
    def __init__(self, data):
        self.data = data["input_ids"]
        self.label = data["labels"]
        self.length = len(data)

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        data = self.data[index]
        label = self.label[index]

        data = torch.tensor(data, dtype=torch.int64)
        label = torch.tensor(label, dtype=torch.int64)

        return (data, label)

In [19]:
torch_train_set = CustomDataset(hf_train_tokenized)
torch_val_set = CustomDataset(hf_val_tokenized)
torch_test_set = CustomDataset(hf_test_tokenized)

In [20]:
torch_train_loader = data.DataLoader(torch_train_set, batch_size=batch_size, shuffle=True)
torch_val_loader = data.DataLoader(torch_val_set, batch_size=batch_size, shuffle=True)
torch_test_loader = data.DataLoader(torch_test_set, batch_size=batch_size, shuffle=True)

In [21]:
def init_lstm_weights(lstm, low=-0.08, high=0.08):
    for name, param in lstm.named_parameters():
        if 'weight' in name:
            nn.init.uniform_(param, low, high)
        elif 'bias' in name:
            nn.init.zeros_(param)

        return lstm

In [22]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, num_layers, drop_out):
        super(Encoder, self).__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.dropout = nn.Dropout(drop_out)
        self.embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_size,
                              padding_idx=0,
                              max_norm=None,
                              norm_type=2.0,
                              scale_grad_by_freq=False,
                              sparse=False)

        self.rnn = nn.LSTM(input_size=embedding_size, hidden_size=hidden_size, num_layers=num_layers, dropout=drop_out)

        self.rnn = init_lstm_weights(self.rnn)

    def forward(self, x):
        # x: input shape (seq_max_len, batch_size)
        #print(x.shape)
        embeddings_pre_dropout = self.embeddings(x)

        #print(embeddings_pre_dropout.shape)

        #print(embeddings_pre_dropout.is_leaf)
        #print(x.is_leaf)
        #print(embeddings_pre_dropout.mean().backward())

        embeddings = self.dropout(embeddings_pre_dropout)
        #print(embeddings.is_leaf)

        output, (hidden, cell) = self.rnn(embeddings)

        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, num_layers, drop_out):
        super(Decoder, self).__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.dropout = nn.Dropout(drop_out)
        self.embeddings = nn.Embedding(vocab_size, embedding_size,
                              padding_idx=0,
                              max_norm=None,
                              norm_type=2.0,
                              scale_grad_by_freq=False,
                              sparse=False)

        self.rnn = nn.LSTM(input_size=embedding_size, hidden_size=hidden_size, num_layers=num_layers, dropout=drop_out)

        self.rnn = init_lstm_weights(self.rnn)

        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden, cell):
        # x: input shape (batch_size)
        x = x.unsqueeze(0)

        embeddings = self.dropout(self.embeddings(x))

        outputs, (hidden, cell) = self.rnn(embeddings, (hidden, cell))

        predictions = self.fc(outputs)

        predictions = predictions.squeeze(0)

        return predictions, hidden, cell

In [23]:
class Seq2SeqEncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder, rev_words_dict):
        super(Seq2SeqEncoderDecoder, self).__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.rev_words_dict = rev_words_dict
        self.vocab_len = len(rev_words_dict.keys())

    def forward(self, source, target=None, teacher_force_ratio=.5):
        # input shape (seq_len, batch_size)
        batch_size = source.shape[1]
        target_len = max_token_length

        #print(f"Target len: {target_len}, batch_size = {batch_size}")

        outputs = torch.zeros(target_len + 1, batch_size, self.vocab_len).to(device=device)
        outputs[0, :, 1] = torch.tensor(1)

        hidden, cell = self.encoder(source)

        x = torch.tensor(1)

        for i in range(0, target_len):
            output, hidden, cell = self.decoder(x, hidden, cell)

            outputs[i] = output

            best_guess = output.argmax(1)

            if target is None:
                x = best_guess
            else:
                x = target[i] if random.random() < teacher_force_ratio else best_guess

        return outputs[1:]

# Transformer


In [24]:
class Embedder(nn.Module):
    def __init__(self, vocab_size, d_model=512):
        super(Embedder, self).__init__()
        self.vocab_size = vocab_size
        self.embed_dim = d_model

        self.embed = nn.Embedding(vocab_size, d_model)

    def forward(self, x):
        out = self.embed(x)
        return out

In [25]:
class PositionalEncoder(nn.Module):
    def __init__(self, seq_len, d_model):
        super(PositionalEncoder, self).__init__()
        self.d_model = d_model

        pe = torch.zeros(seq_len, self.d_model)
        for pos in range(seq_len):
            for i in range(0, self.d_model, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/self.d_model)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1))/self.d_model)))
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)


    def forward(self, x):
        x = x * math.sqrt(self.d_model)
        seq_len = x.size(1)
        x = x + torch.autograd.Variable(self.pe[:, :seq_len], requires_grad=False)
        return x


# test
# a = torch.rand(size=(32, 10, 512))
# net = PositionalEncoder(seq_len=10, d_model=512)
# b = net(a)
# b.requires_grad
# b.shape

In [26]:
class MultiheadAttention(nn.Module):
  def __init__(self, d_model=512, n_head=8):
    super(MultiheadAttention, self).__init__()
    self.d_model = d_model
    self.n_head = n_head
    self.d_k = int(d_model/n_head)

    self.q_matrix = nn.Linear(d_model, d_model)
    self.k_matrix = nn.Linear(d_model, d_model)
    self.v_matrix = nn.Linear(d_model, d_model)
    self.o_matrix = nn.Linear(d_model, d_model)

  def split_head(self, x):
    # x: (128, 32, 512)
    batch_size = x.shape[0]

    x = torch.moveaxis(x, 0, 1) #(32, 128, 512)

    return x.reshape(batch_size, -1, self.n_head, self.d_k).permute(0, 2, 1, 3) #(32, 10, 512) => (32, 10, 8, 64) => (32, 8, 10, 64)

  def forward(self, q, k, v, mask = None):
    """
    q, k, v: (batch_size, seq_len, d_model)
    mask
    """

    batch_size = q.shape[0]

    q = self.q_matrix(q)
    k = self.k_matrix(k)
    v = self.v_matrix(v)

    q, k, v = self.split_head(q), self.split_head(k), self.split_head(v) #(32, 8, 10, 64)
    k = k.transpose(-2, -1) #(32, 8, 64, 10)

    score = torch.matmul(q, k)/math.sqrt(self.d_model) #(32, 8, 10, 10)
    if mask is not None:
      score = score.masked_fill(mask == 0, -1e9)
    score = nn.functional.softmax(score, dim=-1)

    attn_score = torch.matmul(score, v) #(32, 8, 10, 64)
    attn_score = attn_score.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)

    output = self.o_matrix(attn_score)
    return output

# x = torch.rand(size=(32, 10, 512))
# net = MultiheadAttention(d_model=512, n_head=8)
# a = net(k=x, q=x, v=x)
# a.shape

In [27]:
class TransformerBlock(nn.Module):
  def __init__(self, d_model=512, n_head=8, factor=4):
    super(TransformerBlock, self).__init__()

    self.d_model = d_model
    self.n_head = n_head
    self.factor = factor

    self.multihead_attention = MultiheadAttention(d_model, n_head)
    self.norm1 = nn.LayerNorm(d_model)
    self.norm2 = nn.LayerNorm(d_model)

    self.feed_forward = nn.Sequential(
        nn.Linear(d_model, factor*d_model),
        nn.ReLU(),
        nn.Linear(factor*d_model, d_model)
    )

    self.dropout1 = nn.Dropout(0.2)
    self.dropout2 = nn.Dropout(0.2)

  def forward(self, q, k, v):
    """
    x: (batch_size, sequence length, embedded dimension)
    """

    attention_out = self.multihead_attention(q, k, v)
    attention_res_out = attention_out + v
    norm1_out = self.dropout1(self.norm1(attention_res_out))
    fw_out = self.feed_forward(norm1_out)
    fw_res_out = norm1_out + fw_out
    norm2_out = self.dropout2(self.norm2(fw_res_out))

    return norm2_out



x = torch.rand(size=(128, 32, 512))
net = TransformerBlock(d_model=512, n_head=8)
a = net(x, x, x)
print(a.shape)

torch.Size([128, 32, 512])


In [28]:
class TransformerEncoder(nn.Module):
    def __init__(self, seq_len, vocab_size, d_model=512, num_layer=6, factor=4, n_head=8):
        super(TransformerEncoder, self).__init__()
        self.seq_len = seq_len
        self.vocab_size = vocab_size
        self.num_layer = num_layer
        self.factor = factor
        self.n_head = n_head
        self.d_model = d_model

        self.embedding_layer = Embedder(vocab_size=vocab_size, d_model=d_model)
        self.positional_encoder = PositionalEncoder(seq_len=seq_len, d_model=d_model)

        self.layers = nn.ModuleList([TransformerBlock(d_model=d_model, n_head=n_head, factor=factor) for i in range(num_layer)])

    def forward(self, x):
        embed_out = self.embedding_layer(x)
        out = self.positional_encoder(embed_out)
        for layer in self.layers:
            out = layer(out, out, out)

        return out

x = torch.randint(size=(32, 10), low=1, high=100)
net = TransformerEncoder(seq_len=10, vocab_size=200, num_layer=1)
a = net(x)
print(a.shape)

torch.Size([32, 10, 512])


In [29]:
class DecoderBlock(nn.Module):
  def __init__(self, d_model=512, factor=4, n_head=8):
    super(DecoderBlock, self).__init__()
    self.d_model = d_model
    self.factor = factor
    self.n_head = n_head

    self.attention = MultiheadAttention(d_model, n_head=8)
    self.norm = nn.LayerNorm(d_model)
    self.dropout = nn.Dropout(0.2)
    self.transformer_block = TransformerBlock(d_model, factor=factor, n_head=n_head)

  def forward(self, x, k, v, mask):
    attention = self.attention(x, x, x, mask=mask)
    q = self.dropout(self.norm(attention + x))
    out = self.transformer_block(q, k, v)

    return out

In [30]:
class TransformerDecoder(nn.Module):
  def __init__(self, seq_len, vocab_size, d_model = 512, num_layer=2, factor=4, n_head=8):
    super(TransformerDecoder, self).__init__()
    self.vocab_size = vocab_size
    self.d_model = d_model
    self.num_layer = num_layer
    self.factor = factor
    self.n_head = n_head

    self.embedding_layer = Embedder(vocab_size=vocab_size, d_model=d_model)
    self.positional_encoder = PositionalEncoder(seq_len=seq_len, d_model=d_model)

    self.layers = nn.ModuleList([
        DecoderBlock(d_model, factor=factor, n_head=n_head)
        for i in range(self.num_layer)
    ])

    self.fc_out = nn.Linear(d_model, vocab_size)
    self.dropout = nn.Dropout(0.2)

  def forward(self, x, encoder_out, mask):
    x = self.embedding_layer(x)
    x = self.positional_encoder(x)
    x = self.dropout(x)

    for layer in self.layers:
      x = layer(x=x, k=encoder_out, v=encoder_out, mask=mask)

    out = nn.functional.softmax(self.fc_out(x), dim=-1) #batch_size, seq_len, vocab_size

    return out

In [42]:
class Transformer(nn.Module):
  def __init__(self, d_model, src_vocab_size, target_vocab_size, seq_len, num_layer=6, factor=4, n_head=8):
    super(Transformer, self).__init__()
    self.d_model = d_model
    self.src_vocab_size = src_vocab_size
    self.target_vocab_size = target_vocab_size
    self.seq_len = seq_len
    self.num_laye = num_layer
    self.factor = factor
    self.n_head = n_head

    self.encoder = TransformerEncoder(seq_len=seq_len, vocab_size=src_vocab_size, d_model=d_model, num_layer=num_layer, factor=factor, n_head=n_head)
    self.decoder = TransformerDecoder(seq_len=seq_len, vocab_size=target_vocab_size, d_model=d_model, num_layer=num_layer, factor=factor, n_head=n_head)

  def make_target_mask(self, trg):
    device = trg.device
    tgt_mask = (trg != 0).unsqueeze(1).unsqueeze(3).to(device)
    seq_length = trg.size(1)
    nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool().to(device)
    tgt_mask = tgt_mask & nopeak_mask
    return tgt_mask.to(trg.device)

  def decode(self, src):


    batch_size, seq_len = src.shape[0], src.shape[1]
    trg = torch.zeros(size=(batch_size, seq_len), dtype=torch.int)
    trg[:, 0] = 1

    enc_out = self.encoder(src)

    for i in range(1, seq_len):
      trg_mask = self.make_target_mask(trg)
      out = self.decoder(x=trg, encoder_out=enc_out, mask=trg_mask)
      out = out.argmax(-1)[:, i]
      trg[:, i] = out

    return trg

  def forward(self, src, trg):
    trg_mask = self.make_target_mask(trg)
    enc_out = self.encoder(src)

    outputs = self.decoder(trg, enc_out, trg_mask)
    return outputs

# Train

## Initialize, load, save model

In [50]:
def init_model():
    encoder = Encoder(len(vocab_dict.keys()), embeddings_size, hidden_size, num_layers, dropout_rate)
    decoder = Decoder(len(vocab_dict.keys()), embeddings_size, hidden_size, num_layers, dropout_rate)

    model = Transformer(d_model=d_model, src_vocab_size=len(vocab_dict.keys()), target_vocab_size=len(vocab_dict.keys()),
                        seq_len=max_token_length, num_layer=num_layers, factor=factor, n_head=n_head)

    #class_weights = torch.tensor(100 * np.ones(model.vocab_len), dtype=torch.float32)
    #class_weights[esp_words_dict["<pad>"]] = torch.tensor(1e-6, dtype=torch.float32)
    #class_weights[esp_words_dict["<sos>"]] = torch.tensor(1, dtype=torch.float32)
    #class_weights[esp_words_dict["<eos>"]] = torch.tensor(1, dtype=torch.float32)
    #class_weights[esp_words_dict["<unk>"]] = torch.tensor(1, dtype=torch.float32)

    #print(class_weights)

    criterion = nn.CrossEntropyLoss().to(device=device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    return model, criterion, optimizer

def save_model(model, optimizer, epoch, path):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, path)

def load(model, optimizer, path):
    checkpoint = torch.load(path, map_location=torch.device(device))

    print(type(checkpoint["model_state_dict"]))

    model.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    epoch = checkpoint["epoch"]

    return model, optimizer, epoch

## Metrics + Summary

In [51]:
import evaluate

metric = evaluate.load("sacrebleu")

In [52]:
def compute_metrics(dataset, model=None):
    if model is None:
        MODEL_SAVE_PATH = os.path.join(os.getcwd(), "translation_transformers.pt")
        model, _, optimizer = init_model()
        model, _, _ = load(model, optimizer, MODEL_SAVE_PATH)

    model.eval()

    total_pred = []
    total_label = []

    for data, label in dataset:
        data = data.to(device=device)
        label = label.to(device=device)

        data = torch.moveaxis(data, 1, 0)
        label = torch.moveaxis(label, 1, 0)

        prob = model(data, label, 0)
        pred = torch.argmax(prob, dim=2)

        label = torch.moveaxis(label, 0, 1)
        pred = torch.moveaxis(pred, 0, 1)
        pred_decoded = tokenizer.batch_decode(pred, skip_special_tokens=True)
        label_decoded = tokenizer.batch_decode(label, skip_special_tokens=True)
        total_pred.extend(pred_decoded)
        total_label.extend(label_decoded)

        if len(total_pred) != len(total_label):
            print(len(pred_decoded))
            print(len(label_decoded))

    result = metric.compute(predictions=total_pred, references=total_label)
    result = {"bleu": result["score"]}

    return result

In [53]:
compute_metrics(torch_test_loader)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/ML/MachineTranslation/translation_transformers.pt'

In [54]:
def summary(loader, model, criterion):
    num_correct = 0
    num_samples = 0
    total_loss = 0
    loss_epoch = 0
    loss_avg = 0

    model.eval()

    acc = 0

    with torch.no_grad():
        for index, (data, label) in enumerate(loader):
            data = data.to(device=device)
            label = label.to(device=device)

            prob = model(data, label)

            pred = torch.argmax(prob, dim=2)

            current_correct = (pred == label).sum()
            current_size = pred.shape[0] * pred.shape[1]

            num_correct += current_correct
            num_samples += current_size

            prob = torch.moveaxis(prob, (1, 2), (0, 1))
            label = torch.moveaxis(label, 1, 0)

            #print(data.shape)
            #print(label.shape)
            #print(pred.shape)

            loss = criterion(prob, label)

            loss_epoch += loss.item()

        acc = float(num_correct)/float(num_samples) * 100.0
        loss_avg = float(loss_epoch)/float(len(loader))

        bleu_score = compute_metrics(loader, model)
    return acc, loss_avg, bleu_score

## Actual training loop

In [55]:
def train(train_loader, val_loader, num_epochs, batch_print=40):
    train_acc_list = []
    train_loss_list = []

    val_acc_list = []
    val_loss_list = []
    val_bleu_score = []

    cur_epoch = -1

    model, criterion, optimizer = init_model()

    numpy_final_result = [[] for _ in range(20)]

    MODEL_SAVE_PATH = os.path.join(os.getcwd(), "translation_transformers.pt")
    JSON_SAVE_PATH = os.path.join(os.getcwd(), "translation_transformers.json")

    if os.path.exists(MODEL_SAVE_PATH):
        model, optimizer, cur_epoch = load(model, optimizer, path=MODEL_SAVE_PATH)

        #with open(NUMPY_SAVE_PATH, 'rb') as f:
        #    numpy_final_result = pickle.load(f)

        ### LOAD MODEL ###

    for epoch in range(num_epochs):
        if cur_epoch >= epoch:
            continue

        correct_samples = 0
        total_samples = 0

        loss_epoch = 0

        print("----------------------------------------")

        model.train()

        for batch_idx, (data, label) in enumerate(train_loader):
            # Data to CUDA if possible
            data = data.to(device=device)
            label = label.to(device=device)
            print(data.shape)
            print(label.shape)

            #data = torch.moveaxis(data, 1, 0)
            #label = torch.moveaxis(label, 1, 0)
            #print(data.shape)
            #print(label.shape)
            print("*********")

            optimizer.zero_grad()

            prob = model(data, label)
            #prob.requires_grad=True
            prob.retain_grad()

            pred = torch.argmax(prob, dim=2)

            current_correct = (pred == label).sum()
            current_size = pred.shape[0] * pred.shape[1]

            correct_samples += current_correct
            total_samples += current_size

            prob = torch.moveaxis(prob, (1, 2), (0, 1))
            label = torch.moveaxis(label, 1, 0)

            #print(data.shape)
            #print(label.shape)
            #print(pred.shape)
            #print(prob.shape)

            loss = criterion(prob, label)
            loss.retain_grad()
            #loss.requires_grad=True
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)
            #optimizer.requires_grad=True
            optimizer.step()

            loss_epoch += loss.item()

            if batch_idx % batch_print == batch_print - 1:
                print(f"Batch {batch_idx + 1}: Accuracy: {float(current_correct) / float(current_size) * 100.0}")
                print(f"Loss: {float(loss.item())}")
                save_model(model=model, optimizer=optimizer, epoch=epoch, path=MODEL_SAVE_PATH)

        # Validation
        val_acc, val_loss, bleu_score = summary(val_loader, model, criterion)

        train_acc_list.append(float(correct_samples) / float(total_samples + 1e-12) * 100.0)
        train_loss_list.append(float(loss_epoch) / float(len(train_loader)))

        val_acc_list.append(val_acc)
        val_loss_list.append(val_loss)
        val_bleu_score.append(bleu_score)

        #for i in range(20):
        #    numpy_final_result[i].extend(final_result[i])
        #    print(f"Prob for {i + 1}: min {np.min(numpy_final_result[i])}, max: {np.max(numpy_final_result[i])}")

        if epoch % 1 == 0:
            save_model(model=model, optimizer=optimizer, epoch=epoch, path=MODEL_SAVE_PATH)

        cur_epoch = epoch

        print(f"Epoch {epoch + 1}:")

        print(f"Train accuracy: {train_acc_list[-1]}%")
        print(f"Train loss: {train_loss_list[-1]}")

        print(f"Val accuracy: {val_acc_list[-1]}%")
        print(f"Val loss: {val_loss_list[-1]}")
        print(f"Val bleu score: {val_bleu_score[-1]}")

In [ ]:
train(torch_train_loader, torch_val_loader, num_epochs=epochs, batch_print=50)

----------------------------------------
torch.Size([64, 128])
torch.Size([64, 128])
*********
torch.Size([64, 128])
torch.Size([64, 128])
torch.Size([64, 1, 128, 128])
torch.Size([64, 128, 512])
torch.Size([64, 128, 32100])


KeyboardInterrupt: 

# Evaluation + Inference

In [ ]:
compute_metrics(torch_test_loader)

In [ ]:
text = "I have a cat"

In [ ]:
def inference_directly(text):
    # Tokenize the input text
    inputs = tokenizer(text, padding="max_length", max_length=max_token_length, return_tensors="pt")

    MODEL_SAVE_PATH = os.path.join(os.getcwd(), "translation_transformers.pt")
    model, _, optimizer = init_model()
    model, _, _ = load(model, optimizer, MODEL_SAVE_PATH)

    print(inputs["input_ids"].shape)

    # Generate predictions
    with torch.no_grad():
        outputs = model(inputs["input_ids"].squeeze().unsqueeze(1))

    decoded_preds = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded_preds

In [ ]:
inference_directly(text)